Analysis assingment 
1) The demographics of the game are as follows: 
    Most players are men, being around 
    As well most players are aged ___
2)

In [ ]:
#Import pandas 
import pandas as pd 
import os 

# Load file
csv_path = os.path.join("Resources/Purchase_data.csv")

# Read Purchasing File and store into Pandas data frame
purchase_dataf = pd.read_csv(csv_path)
purchase_dataf

In [ ]:
#Player count 
#Count the total number of players
players = len(purchase_dataf["SN"].unique())
#Create tiny data frame ( more like data box)
Player_countdf = pd.DataFrame({"Total Players":[players]})
Player_countdf

In [ ]:
#Purchasing Analysis (Total)
#Obtain number of unique items, average price, etc. using basic calculations

#Create unique items
items = purchase_dataf["Item Name"].unique()
unique_items = len(items)
#Do average of price column
average_price = round(purchase_dataf["Price"].mean(),2)
#Create number of purchases using purchase ID
tot_purchase_num = len(purchase_dataf["Purchase ID"])
#Create total revenue
tot_revenue = purchase_dataf["Price"].sum()
#Create a new dataframe with new information

purchase_analysisdf = pd.DataFrame({"Unique Items":[unique_items], 
                                    "Average Price": [average_price], 
                                    "Total Purchase Number": [tot_purchase_num], 
                                    "Total Revenue": [tot_revenue]
    
    
})
# Add formatting 
purchase_analysisdf["Average Price"] = purchase_analysisdf["Average Price"].map("${:.2f}".format)
purchase_analysisdf["Total Revenue"] = purchase_analysisdf["Total Revenue"].map("${:.2f}".format)

purchase_analysisdf

In [ ]:
#Gender of players
#Percentage and Count of Male, Female and Other/Non-disclosed players
#Drop players that are more than once
gender_df = purchase_dataf[["SN","Gender"]].drop_duplicates()


gender_count = gender_df["Gender"].value_counts()


#Check why values are different from gitlab
#some players are the same because they made different purchases, change code to get repeated players out of gender count.


In [ ]:
#Get counts to get the percentage 
tot_all = len(gender_df)
tot_males = len(gender_df.loc[gender_df["Gender"] == "Male"])
tot_females = len(gender_df.loc[gender_df["Gender"] == "Female"])
tot_other = len(gender_df.loc[gender_df["Gender"] == "Other / Non-Disclosed"])


In [ ]:
#get percent by gender 
male_pct = round((tot_males/tot_all),2)
female_pct= round((tot_females/tot_all),2)
other_pct = round((tot_other/tot_all),2)

In [ ]:
#make new dataframe to display the results for gender of players
Final_genderdf = pd.DataFrame({"Gender":["Male","Female","Other / Non-Disclosed"], 
                                    "Total Count": [tot_males,tot_females,tot_other], 
                                    "Percentage of Players": ["{:.2%}".format(male_pct),
                                                              "{:.2%}".format(female_pct),"{:.2%}".format(other_pct)]  
})
Gender_countdf = Final_genderdf.set_index("Gender")
Gender_countdf 


In [ ]:
#Purchasing analysis by gender 
#Use groupby 
Gender_group = purchase_dataf.groupby(["Gender"])

#Obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
Gender_summarydf = Gender_group.Price.agg(["count", "sum", "mean"])
Gender_summarydf

#Get average purchase total per person
#Bring data on total count of players
Purchase_per_player = round((Gender_summarydf["sum"]/Gender_countdf["Total Count"]),2)
Purchase_per_player


# Change de names of the columns of dataframe 
Gender_summarydf = Gender_summarydf.rename(columns={"count": "Purchase Count",
                                                  "sum": "Total Purchase Value",
                                                   "mean": "Average Purchase Value"})


#Add purchase per player column to the end of the dataframe.
Gender_summarydf["Avg Purchase per Player"] = Purchase_per_player

#Give formatting 
Gender_summarydf["Total Purchase Value"] = Gender_summarydf["Total Purchase Value"].map("${:.2f}".format)
Gender_summarydf["Average Purchase Value"] = Gender_summarydf["Average Purchase Value"].map("${:.2f}".format)
Gender_summarydf["Avg Purchase per Player"] = Gender_summarydf["Avg Purchase per Player"].map("${:.2f}".format)

#Create a summary data frame and display it
Gender_summarydf

#.astype(float).map("${:,.2f}".format)

In [ ]:
#Age Demographics
# Drop duplicates for age 
age_df = purchase_dataf[["SN","Age"]].drop_duplicates()

#Make bins for ages. Put players in corresponding category using the bins (Hint use: use pd.cut())

bins =[0,9,14,19,24,29,34,39,150]

named_bins = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

age_df["Age Group"] = pd.cut(age_df["Age"], bins, labels=named_bins, include_lowest=True)




In [ ]:
#Use groupby to group by age 
age_groupdf = age_df.groupby(["Age Group"])
# Calculate the numbers and percentages by age group
get_SN = age_groupdf.count()
SN_total = get_SN["SN"]

Age_pct = ((SN_total*100)/576)
get_SN ["Player Percent"] = Age_pct.astype(float).map("{:,.2f}%".format)
#Create a summary data frame and display the results
get_SN

In [ ]:
#Purchasing Analysis (Age)
#Make new dataframe for purchase and drop SN duplicates
Age_purchase = purchase_dataf[["SN","Age", "Price"]].drop_duplicates()

# Use bins made above and add them to the new data frame
Age_purchase["Age Group"] = pd.cut(Age_purchase["Age"], bins, labels=named_bins, include_lowest=True)

# Make groups with groupby
Age_purchase_group = Age_purchase.groupby(["Age Group"])


#Code for purchase count, avg. purchase price, avg. purchase total per person etc.
#Total Number of Items Purchased
age_total_items =Age_purchase_group["Price"].count()

#Average Purchase Price
age_avg_price = Age_purchase_group["Price"].mean()



#Total Revenue

age_tot_revenue = Age_purchase_group["Price"].sum()

#Average per person per age 
# I already know that the player total by age was already done before
Purchase_per_age = round((age_tot_revenue/get_SN["Age"]),2)


In [ ]:
#Make new dataframe to show results 
Final_agedf = pd.DataFrame({ "Purchase count":age_total_items.map("${:,.2f}".format), 
                                    "Average Purchase Price": age_avg_price.map("${:,.2f}".format),
                                   "Total Purchase Value":age_tot_revenue.map("${:,.2f}".format),
                                    "Avg Total Purchase per Person":Purchase_per_age.map("${:,.2f}".format)
})
Final_agedf

In [ ]:
#Top Spenders
#Discover who is the top spender, and sort results in descending order. 

#Separate SN (player with their purchases)

#Create dataframe for spenders 

Spenderdf = purchase_dataf[["SN","Price","Purchase ID"]]

#groupby SN to get players
purch_by_player = Spenderdf.groupby("SN")

#Make purchase count 
num_purchase = purch_by_player["Purchase ID"].count()

#SUm values to get total purchase values
tot_purch_value = purch_by_player["Price"].sum()

#Average purchase value by spender 
avg_purch_value =purch_by_player["Price"].mean()

In [ ]:
#Create the new dataframe with spender data 
Final_spenderdf = pd.DataFrame({ "Purchase Count ": num_purchase,
                                    "Average Purchase Price": avg_purch_value ,
                                   "Total Purchase Value": tot_purch_value
})

#apply ascending = false 
Final_spenderdf = Final_spenderdf.sort_values("Total Purchase Value", ascending = False)


#Format in the end to avoid mistakes 
Final_spenderdf["Average Purchase Price"] = Final_spenderdf["Average Purchase Price"].map("${:,.2f}".format)
Final_spenderdf["Total Purchase Value"] = Final_spenderdf["Total Purchase Value"].map("${:,.2f}".format)
Final_spenderdf.head()

In [ ]:
#Most Popular Items

#Calculate purchase count, average item price, and total purchase value
#apply ascending = false 



#Retrieve the Item ID, Item Name, and Item Price columns. 
popular_items = purchase_dataf[["Item ID", "Item Name", "Price"]]

#Group Item ID and Item name
pop_items_group = popular_items.groupby(["Item ID", "Item Name"])

#Count how many items have been bought of each
pop_item_count= pop_items_group["Price"].count()

#Get Item price average. 
pop_item_priceavg = pop_items_group["Price"].mean()

#Get total purchase value 
tot_purchase_item = pop_items_group["Price"].sum()




In [ ]:
#Create new dataframe to display data 
popular_item_final = pd.DataFrame({"Purchase Count": pop_item_count,
                                 "Item Price": pop_item_priceavg,
                                 "Total Purchase Value": tot_purchase_item})

#Sort values, format later 
popular_item_final = popular_item_final.sort_values(["Purchase Count","Item ID"], ascending=False)

profit_item = popular_item_final.copy()

#Format in the end to avoid mistakes
popular_item_final["Item Price"] = popular_item_final["Item Price"].map("${:,.2f}".format)
popular_item_final["Total Purchase Value"] = popular_item_final["Total Purchase Value"].map("${:,.2f}".format)

popular_item_final.head()

In [ ]:
#Most Profitable Items

#groupby Items
#count those items 

In [ ]:
#Sort the above table by total purchase value in descending order. Create and display new dataframe